In [1]:
import os
import torch
import rasterio
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
from torchvision.transforms import functional as TF
from torch.utils.data import Dataset, DataLoader

class SuperResolutionDataset(Dataset):
    def __init__(self, large_tiff_dir, patch_size=4, stride=None, padding_ratio=0.25,
                 s2_160m_name='s2_20_x8.tif', s2_80m_name='s2_10_x8.tif',
                 planet_20m_name='planet_x8.tif', label_name='s2_20.tif',
                 s2_160m_mask_name='mask_20_x8.tif', s2_80m_mask_name='mask_10_x8.tif',
                 regions=['jiri_1', 'jiri_2', 'sobaek']):
        """
        Super-resolution을 위한 데이터셋 클래스 (이미지 및 마스크 캐싱 적용)
        """
        self.large_tiff_dir = large_tiff_dir
        self.patch_size_160m = patch_size
        self.patch_size_80m = patch_size * 2
        self.patch_size_20m = patch_size * 8
        
        self.stride = stride if stride is not None else patch_size
        self.padding_160m = int(padding_ratio * patch_size)
        self.padding_80m = self.padding_160m * 2
        self.padding_20m = self.padding_160m * 8
        
        self.s2_160m_name = s2_160m_name
        self.s2_80m_name = s2_80m_name
        self.planet_20m_name = planet_20m_name
        self.label_name = label_name
        self.s2_160m_mask_name = s2_160m_mask_name
        self.s2_80m_mask_name = s2_80m_mask_name
        
        self.regions = regions
        
        # 이미지 캐시 (메모리에 한 번만 로드)
        self.image_cache = {}
        self._cache_images()
        
        # 인덱스 리스트 생성
        self.index_list = self._create_index_list()

    def _cache_images(self):
        """TIFF 이미지를 메모리에 캐싱"""
        for region in self.regions:
            self.image_cache[region] = {}
            for img_name in [self.s2_160m_name, self.s2_80m_name, self.planet_20m_name, self.label_name, 
                             self.s2_160m_mask_name, self.s2_80m_mask_name]:
                path = os.path.join(self.large_tiff_dir, region, img_name)
                with rasterio.open(path) as src:
                    self.image_cache[region][img_name] = src.read()
    
    
    def _create_index_list(self):
        """모든 지역의 패치 인덱스를 생성하여 리스트에 저장"""
        index_list = []
        for region in self.regions:
            h, w = self.image_cache[region][self.s2_160m_name].shape[1:]  # (채널, 높이, 너비)

            num_patches_h = (h - self.patch_size_160m) // self.stride + 1
            num_patches_w = (w - self.patch_size_160m) // self.stride + 1

            for patch_idx in range(num_patches_h * num_patches_w):
                y = (patch_idx // num_patches_w) * self.stride
                x = (patch_idx % num_patches_w) * self.stride
                index_list.append((region, x, y))

        return index_list

    def _apply_padding(self, img, padding):
        """Reflection Padding 적용"""
        return F.pad(torch.tensor(img, dtype=torch.float32),
                     (padding, padding, padding, padding), mode='reflect')
    
    def __len__(self):
        return len(self.index_list)
    
    def __getitem__(self, idx):
        region, x, y = self.index_list[idx]

        # 캐싱된 이미지에서 데이터 추출
        s2_160m = self.image_cache[region][self.s2_160m_name]
        s2_80m = self.image_cache[region][self.s2_80m_name]
        planet_20m = self.image_cache[region][self.planet_20m_name]
        label_20m = self.image_cache[region][self.label_name]
        s2_160m_mask = self.image_cache[region][self.s2_160m_mask_name]
        s2_80m_mask = self.image_cache[region][self.s2_80m_mask_name]

        # 패치 추출
        s2_160m_patch = s2_160m[:, y:y+self.patch_size_160m, x:x+self.patch_size_160m]
        s2_80m_patch = s2_80m[:, y*2:y*2+self.patch_size_80m, x*2:x*2+self.patch_size_80m]
        planet_20m_patch = planet_20m[:, y*8:y*8+self.patch_size_20m, x*8:x*8+self.patch_size_20m]
        label_patch = label_20m[:, y*8:y*8+self.patch_size_20m, x*8:x*8+self.patch_size_20m]
        s2_160m_mask_patch = s2_160m_mask[:, y:y+self.patch_size_160m, x:x+self.patch_size_160m]
        s2_80m_mask_patch = s2_80m_mask[:, y*2:y*2+self.patch_size_80m, x*2:x*2+self.patch_size_80m]

        # 패딩 적용
        s2_160m_patch = self._apply_padding(s2_160m_patch, self.padding_160m)
        s2_80m_patch = self._apply_padding(s2_80m_patch, self.padding_80m)
        planet_20m_patch = self._apply_padding(planet_20m_patch, self.padding_20m)
        label_patch = self._apply_padding(label_patch, self.padding_20m)
        s2_160m_mask_patch = self._apply_padding(s2_160m_mask_patch, self.padding_160m)
        s2_80m_mask_patch = self._apply_padding(s2_80m_mask_patch, self.padding_80m)

        # 반환
        return (s2_160m_patch, s2_160m_mask_patch, s2_80m_patch, s2_80m_mask_patch ,planet_20m_patch), label_patch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SubPixelBlock(nn.Module):
    def __init__(self, in_channels, scale_factor):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, in_channels * (scale_factor ** 2), kernel_size=3, padding=1)
        self.pixel_shuffle = nn.PixelShuffle(scale_factor)
        self.prelu = nn.PReLU()

    def forward(self, x):
        x = self.conv(x)  # 채널 확장
        x = self.pixel_shuffle(x)  # 공간 해상도 증가
        return self.prelu(x)

class ResNetBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(in_channels)
        self.prelu = nn.PReLU()
        self.conv2 = nn.Conv2d(in_channels, in_channels, kernel_size=3, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(in_channels)

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.prelu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        return out + residual  # Skip Connection

class SuperResolutionResNet(nn.Module):
    def __init__(self, in_channels_160m=7, in_channels_80m=5, in_channels_planet=4, num_blocks=10):
        super().__init__()
        self.subpixel_160m = SubPixelBlock(in_channels_160m, scale_factor=8)
        self.subpixel_80m = SubPixelBlock(in_channels_80m, scale_factor=4)
        
        total_channels = in_channels_160m + in_channels_80m + in_channels_planet
        
        ################################
        self.conv_160m_1 = nn.Conv2d(in_channels_160m, in_channels_160m, kernel_size=3, padding=1)
        self.conv_160m_2 = nn.Conv2d(in_channels_160m, in_channels_160m, kernel_size=3, padding=1)
        
        self.conv_80m_1 = nn.Conv2d(in_channels_80m, in_channels_80m, kernel_size=3, padding=1)
        self.conv_80m_2 = nn.Conv2d(in_channels_80m, in_channels_80m, kernel_size=3, padding=1)
        
        self.conv_planet_1 = nn.Conv2d(in_channels_planet, in_channels_planet, kernel_size=3, padding=1)
        self.conv_planet_2 = nn.Conv2d(in_channels_planet, in_channels_planet, kernel_size=3, padding=1)
        ################################
        
        self.initial_conv = nn.Conv2d(total_channels, total_channels, kernel_size=3, padding=1)
        self.res_blocks = nn.Sequential(*[ResNetBlock(total_channels) for _ in range(num_blocks)])
        self.final_conv = nn.Conv2d(total_channels, in_channels_160m-1, kernel_size=3, padding=1)
    
    def forward(self, s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m):
        x_160m = torch.cat([s2_160m, s2_160m_mask], dim=1)
        x_80m = torch.cat([s2_80m, s2_80m_mask], dim=1)
        
        #################################
        x_160m = self.conv_160m_1(x_160m) + x_160m
        x_160m = self.conv_160m_2(x_160m) + x_160m
        
        x_80m = self.conv_80m_1(x_80m) + x_80m
        x_80m = self.conv_80m_2(x_80m) + x_80m
        
        planet_20m = self.conv_planet_1(planet_20m) + planet_20m
        planet_20m = self.conv_planet_2(planet_20m) + planet_20m
        ################################
        
        x_160m = self.subpixel_160m(x_160m)
        x_80m = self.subpixel_80m(x_80m)
        
        x = torch.cat([x_160m, x_80m, planet_20m], dim=1)
        x = self.initial_conv(x)
        x = self.res_blocks(x)
        x = self.final_conv(x)
        
        return x

In [3]:
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR
from tqdm import tqdm

# 학습 루프 정의
def train_one_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    epoch_loss = 0
    
    for data in tqdm(dataloader, desc='Training', leave=False):
        x, target = data
        s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = x
        
        s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m, target = (
            s2_160m.to(device), s2_160m_mask.to(device), s2_80m.to(device), 
            s2_80m_mask.to(device), planet_20m.to(device), target.to(device)
        )
        
        optimizer.zero_grad()
        output = model(s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m)
        loss = criterion(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        epoch_loss += loss.item()
    
    return epoch_loss / len(dataloader)

def train(model, train_loader, val_loader, epochs=50, lr=1e-4, device='cuda'):
    model.to(device)
    criterion = nn.L1Loss()
    optimizer = AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = StepLR(optimizer, step_size=20, gamma=0.5)
    
    for epoch in range(epochs):
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, device)
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{epochs} - Train Loss: {train_loss:.6f}")

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS

device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )
epochs = 100

output_base_dir = "../../data/high_resolution_data" #결과 저장할 디렉토리
# output_base_dir = "./" #결과 저장할 디렉토리

os.makedirs(output_base_dir, exist_ok=True)

meta_data_dict = {
    'jiri_1' : {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 371940.0,
       0.0, -2.5, 3915480.0)},

    'jiri_2': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 380940.0,
       0.0, -2.5, 3915480.0)},

    'sobaek': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 449760.0,
       0.0, -2.5, 4096740.0)}}

org_patch_size = 32
org_stride = 8 # (456(원래 데이터 크기) - 32) % stride == 0 이어야함

df = pd.read_csv("../../data/super_resolution_data_scaled/percentile_values.csv")

# 전체 해상도 설정
full_size = (6, 3648, 3648) # (채널, 높이, 너비)
patch_size = 8*org_patch_size # 패딩 제거 후 남는 유효 영역
stride = 8*org_stride # 데이터셋의 stride 값


for period in range(1,13,1):
    print(f"{period} 시작")
    
    #데이터 셋 만들기
    train_dataset = SuperResolutionDataset(f"../../data/super_resolution_data_scaled/{period}", stride=2)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    
    #모델 생성
    model = SuperResolutionResNet(num_blocks=25)
    total_parameters = sum(p.numel() for p in model.parameters())
    print(f"전체 파라미터 수: {total_parameters}")
    
    #학습
    train(model, train_loader, None, epochs=epochs, lr=1e-4, device=device)
    
    #메모리 확보
    del train_loader, train_dataset
    torch.cuda.empty_cache()
    
    for region in ['jiri_1', 'jiri_2', 'sobaek']:
        # CPU에서 처리할 NumPy 배열로 변경
        total_output = np.zeros(full_size, dtype=np.float32)
        total_count = np.zeros((1, full_size[1], full_size[2]), dtype=np.float32)
        high_resolution_data = np.zeros(full_size, dtype=np.float32) #최종 결과

        # 추론용 데이터 로더 준비
        inference_dataset = SuperResolutionDataset(
            f"../../data/super_resolution_data_scaled/{period}",
            patch_size=org_patch_size,
            s2_160m_name='s2_20.tif', s2_80m_name='s2_10.tif',
            planet_20m_name='planet.tif', label_name='planet.tif',
            s2_160m_mask_name='mask_20.tif', s2_80m_mask_name='mask_10.tif',
            stride=org_stride, regions=[region]
        )

        inference_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)

        # 시작 좌표 설정
        patch_idx = 0

        for data in tqdm(inference_loader, desc='inferencing', leave=False):
            x, _ = data
            s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = x
            s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = (
                s2_160m.to(device), s2_160m_mask.to(device), s2_80m.to(device),
                s2_80m_mask.to(device), planet_20m.to(device)
            )

            # 모델 추론
            output = model(s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m)
            output = output[0, :, 64:320, 64:320].detach().cpu().numpy()

            y = (patch_idx // ((full_size[1] - patch_size) // stride + 1)) * stride
            x = (patch_idx % ((full_size[2] - patch_size) // stride + 1)) * stride

            total_output[:, y:y+patch_size, x:x+patch_size] += output
            total_count[:, y:y+patch_size, x:x+patch_size] += 1
            
            del s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m, output
            torch.cuda.empty_cache()
            
            patch_idx+=1

        total_output = total_output / total_count #겹쳐진 수 대로 나눔

        high_resolution_data = np.zeros(full_size, dtype=np.float32)
        for band in range(total_output.shape[0]):
            p_1st = df[(df['period'] == period) & (df['image_type'] == 's2_20') & (df['band'] == band)]['1st_percentile'].values[0].astype('float32')
            p_99th = df[(df['period'] == period) & (df['image_type'] == 's2_20') & (df['band'] == band)]['99th_percentile'].values[0].astype('float32')
            r = p_99th - p_1st
            high_resolution_data[band, :, :] = (total_output[band, :, :]* r) + p_1st

        high_resolution_data = high_resolution_data[:,24:3624, 24:3624] #패딩 걷어 내기

        # 해당 지역의 메타데이터 불러오기
        meta = meta_data_dict[region]

        # 저장을 위한 메타데이터 업데이트
        meta.update({
            'height': high_resolution_data.shape[1],
            'width': high_resolution_data.shape[2],
            'count': high_resolution_data.shape[0],
            'dtype': 'float32'
        })

        # 저장할 파일 경로 설정
        region_dir = os.path.join(output_base_dir, region)
        os.makedirs(region_dir, exist_ok=True)
        
        period_dir = os.path.join(region_dir, f"{period}")
        os.makedirs(period_dir, exist_ok=True)
        
        output_tiff_path = os.path.join(period_dir, 's2.tif')

        # TIFF 파일로 저장
        with rasterio.open(output_tiff_path, 'w', **meta) as dst:
            dst.write(high_resolution_data)
            
        print(f"{output_tiff_path}저장 완료")
        
        del total_output, total_count, high_resolution_data
        torch.cuda.empty_cache()
        
    del inference_loader, inference_dataset, model
    torch.cuda.empty_cache()


1 시작
전체 파라미터 수: 154021


Epoch 1/100 - Train Loss: 0.158551


Epoch 2/100 - Train Loss: 0.097798


Epoch 3/100 - Train Loss: 0.088388


Epoch 4/100 - Train Loss: 0.083573


...

../../data/high_resolution_data/jiri_2/7/s2.tif저장 완료


../../data/high_resolution_data/sobaek/7/s2.tif저장 완료
8 시작
전체 파라미터 수: 154021


Epoch 1/100 - Train Loss: 0.179387


Epoch 2/100 - Train Loss: 0.080103


Training:  21%|████████████████████████████████████████▌                                                                                                                                                        | 460/2187 [00:09<00:36, 47.08it/s]

In [4]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS

device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )
epochs = 100

output_base_dir = "../../data/high_resolution_data" #결과 저장할 디렉토리
# output_base_dir = "./" #결과 저장할 디렉토리

os.makedirs(output_base_dir, exist_ok=True)

meta_data_dict = {
    'jiri_1' : {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 371940.0,
       0.0, -2.5, 3915480.0)},

    'jiri_2': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 380940.0,
       0.0, -2.5, 3915480.0)},

    'sobaek': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 449760.0,
       0.0, -2.5, 4096740.0)}}

org_patch_size = 32
org_stride = 8 # (456(원래 데이터 크기) - 32) % stride == 0 이어야함

df = pd.read_csv("../../data/super_resolution_data_scaled/percentile_values.csv")

# 전체 해상도 설정
full_size = (6, 3648, 3648) # (채널, 높이, 너비)
patch_size = 8*org_patch_size # 패딩 제거 후 남는 유효 영역
stride = 8*org_stride # 데이터셋의 stride 값


for period in range(8,13,1):
    print(f"{period} 시작")
    
    #데이터 셋 만들기
    train_dataset = SuperResolutionDataset(f"../../data/super_resolution_data_scaled/{period}", stride=2)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    
    #모델 생성
    model = SuperResolutionResNet(num_blocks=25)
    total_parameters = sum(p.numel() for p in model.parameters())
    print(f"전체 파라미터 수: {total_parameters}")
    
    #학습
    train(model, train_loader, None, epochs=epochs, lr=1e-4, device=device)
    
    #메모리 확보
    del train_loader, train_dataset
    torch.cuda.empty_cache()
    
    for region in ['jiri_1', 'jiri_2', 'sobaek']:
        # CPU에서 처리할 NumPy 배열로 변경
        total_output = np.zeros(full_size, dtype=np.float32)
        total_count = np.zeros((1, full_size[1], full_size[2]), dtype=np.float32)
        high_resolution_data = np.zeros(full_size, dtype=np.float32) #최종 결과

        # 추론용 데이터 로더 준비
        inference_dataset = SuperResolutionDataset(
            f"../../data/super_resolution_data_scaled/{period}",
            patch_size=org_patch_size,
            s2_160m_name='s2_20.tif', s2_80m_name='s2_10.tif',
            planet_20m_name='planet.tif', label_name='planet.tif',
            s2_160m_mask_name='mask_20.tif', s2_80m_mask_name='mask_10.tif',
            stride=org_stride, regions=[region]
        )

        inference_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)

        # 시작 좌표 설정
        patch_idx = 0

        for data in tqdm(inference_loader, desc='inferencing', leave=False):
            x, _ = data
            s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = x
            s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = (
                s2_160m.to(device), s2_160m_mask.to(device), s2_80m.to(device),
                s2_80m_mask.to(device), planet_20m.to(device)
            )

            # 모델 추론
            output = model(s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m)
            output = output[0, :, 64:320, 64:320].detach().cpu().numpy()

            y = (patch_idx // ((full_size[1] - patch_size) // stride + 1)) * stride
            x = (patch_idx % ((full_size[2] - patch_size) // stride + 1)) * stride

            total_output[:, y:y+patch_size, x:x+patch_size] += output
            total_count[:, y:y+patch_size, x:x+patch_size] += 1
            
            del s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m, output
            torch.cuda.empty_cache()
            
            patch_idx+=1

        total_output = total_output / total_count #겹쳐진 수 대로 나눔

        high_resolution_data = np.zeros(full_size, dtype=np.float32)
        for band in range(total_output.shape[0]):
            p_1st = df[(df['period'] == period) & (df['image_type'] == 's2_20') & (df['band'] == band)]['1st_percentile'].values[0].astype('float32')
            p_99th = df[(df['period'] == period) & (df['image_type'] == 's2_20') & (df['band'] == band)]['99th_percentile'].values[0].astype('float32')
            r = p_99th - p_1st
            high_resolution_data[band, :, :] = (total_output[band, :, :]* r) + p_1st

        high_resolution_data = high_resolution_data[:,24:3624, 24:3624] #패딩 걷어 내기

        # 해당 지역의 메타데이터 불러오기
        meta = meta_data_dict[region]

        # 저장을 위한 메타데이터 업데이트
        meta.update({
            'height': high_resolution_data.shape[1],
            'width': high_resolution_data.shape[2],
            'count': high_resolution_data.shape[0],
            'dtype': 'float32'
        })

        # 저장할 파일 경로 설정
        region_dir = os.path.join(output_base_dir, region)
        os.makedirs(region_dir, exist_ok=True)
        
        period_dir = os.path.join(region_dir, f"{period}")
        os.makedirs(period_dir, exist_ok=True)
        
        output_tiff_path = os.path.join(period_dir, 's2.tif')

        # TIFF 파일로 저장
        with rasterio.open(output_tiff_path, 'w', **meta) as dst:
            dst.write(high_resolution_data)
            
        print(f"{output_tiff_path}저장 완료")
        
        del total_output, total_count, high_resolution_data
        torch.cuda.empty_cache()
        
    del inference_loader, inference_dataset, model
    torch.cuda.empty_cache()


8 시작
전체 파라미터 수: 154021


Epoch 1/100 - Train Loss: 0.189831


Epoch 2/100 - Train Loss: 0.079114


Epoch 3/100 - Train Loss: 0.066419


Epoch 4/100 - Train Loss: 0.058510


...

Epoch 2/100 - Train Loss: 0.080706


Epoch 3/100 - Train Loss: 0.072897


Epoch 4/100 - Train Loss: 0.068369


Epoch 5/100 - Train Loss: 0.065205


KeyboardInterrupt: 

In [ ]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS

device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )
epochs = 100

output_base_dir = "../../data/high_resolution_data" #결과 저장할 디렉토리
# output_base_dir = "./" #결과 저장할 디렉토리

os.makedirs(output_base_dir, exist_ok=True)

meta_data_dict = {
    'jiri_1' : {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 371940.0,
       0.0, -2.5, 3915480.0)},

    'jiri_2': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 380940.0,
       0.0, -2.5, 3915480.0)},

    'sobaek': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 449760.0,
       0.0, -2.5, 4096740.0)}}

org_patch_size = 32
org_stride = 8 # (456(원래 데이터 크기) - 32) % stride == 0 이어야함

df = pd.read_csv("../../data/super_resolution_data_scaled/percentile_values.csv")

# 전체 해상도 설정
full_size = (6, 3648, 3648) # (채널, 높이, 너비)
patch_size = 8*org_patch_size # 패딩 제거 후 남는 유효 영역
stride = 8*org_stride # 데이터셋의 stride 값


for period in range(9,13,1):
    print(f"{period} 시작")
    
    #데이터 셋 만들기
    train_dataset = SuperResolutionDataset(f"../../data/super_resolution_data_scaled/{period}", stride=2)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    
    #모델 생성
    model = SuperResolutionResNet(num_blocks=25)
    total_parameters = sum(p.numel() for p in model.parameters())
    print(f"전체 파라미터 수: {total_parameters}")
    
    #학습
    train(model, train_loader, None, epochs=epochs, lr=1e-4, device=device)
    
    #메모리 확보
    del train_loader, train_dataset
    torch.cuda.empty_cache()
    
    for region in ['jiri_1', 'jiri_2', 'sobaek']:
        # CPU에서 처리할 NumPy 배열로 변경
        total_output = np.zeros(full_size, dtype=np.float32)
        total_count = np.zeros((1, full_size[1], full_size[2]), dtype=np.float32)
        high_resolution_data = np.zeros(full_size, dtype=np.float32) #최종 결과

        # 추론용 데이터 로더 준비
        inference_dataset = SuperResolutionDataset(
            f"../../data/super_resolution_data_scaled/{period}",
            patch_size=org_patch_size,
            s2_160m_name='s2_20.tif', s2_80m_name='s2_10.tif',
            planet_20m_name='planet.tif', label_name='planet.tif',
            s2_160m_mask_name='mask_20.tif', s2_80m_mask_name='mask_10.tif',
            stride=org_stride, regions=[region]
        )

        inference_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)

        # 시작 좌표 설정
        patch_idx = 0

        for data in tqdm(inference_loader, desc='inferencing', leave=False):
            x, _ = data
            s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = x
            s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = (
                s2_160m.to(device), s2_160m_mask.to(device), s2_80m.to(device),
                s2_80m_mask.to(device), planet_20m.to(device)
            )

            # 모델 추론
            output = model(s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m)
            output = output[0, :, 64:320, 64:320].detach().cpu().numpy()

            y = (patch_idx // ((full_size[1] - patch_size) // stride + 1)) * stride
            x = (patch_idx % ((full_size[2] - patch_size) // stride + 1)) * stride

            total_output[:, y:y+patch_size, x:x+patch_size] += output
            total_count[:, y:y+patch_size, x:x+patch_size] += 1
            
            del s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m, output
            torch.cuda.empty_cache()
            
            patch_idx+=1

        total_output = total_output / total_count #겹쳐진 수 대로 나눔

        high_resolution_data = np.zeros(full_size, dtype=np.float32)
        for band in range(total_output.shape[0]):
            p_1st = df[(df['period'] == period) & (df['image_type'] == 's2_20') & (df['band'] == band)]['1st_percentile'].values[0].astype('float32')
            p_99th = df[(df['period'] == period) & (df['image_type'] == 's2_20') & (df['band'] == band)]['99th_percentile'].values[0].astype('float32')
            r = p_99th - p_1st
            high_resolution_data[band, :, :] = (total_output[band, :, :]* r) + p_1st

        high_resolution_data = high_resolution_data[:,24:3624, 24:3624] #패딩 걷어 내기

        # 해당 지역의 메타데이터 불러오기
        meta = meta_data_dict[region]

        # 저장을 위한 메타데이터 업데이트
        meta.update({
            'height': high_resolution_data.shape[1],
            'width': high_resolution_data.shape[2],
            'count': high_resolution_data.shape[0],
            'dtype': 'float32'
        })

        # 저장할 파일 경로 설정
        region_dir = os.path.join(output_base_dir, region)
        os.makedirs(region_dir, exist_ok=True)
        
        period_dir = os.path.join(region_dir, f"{period}")
        os.makedirs(period_dir, exist_ok=True)
        
        output_tiff_path = os.path.join(period_dir, 's2.tif')

        # TIFF 파일로 저장
        with rasterio.open(output_tiff_path, 'w', **meta) as dst:
            dst.write(high_resolution_data)
            
        print(f"{output_tiff_path}저장 완료")
        
        del total_output, total_count, high_resolution_data
        torch.cuda.empty_cache()
        
    del inference_loader, inference_dataset, model
    torch.cuda.empty_cache()


9 시작
전체 파라미터 수: 154021


Epoch 1/100 - Train Loss: 0.138137


Epoch 2/100 - Train Loss: 0.080544


Epoch 3/100 - Train Loss: 0.073517


Epoch 4/100 - Train Loss: 0.069155


...

Epoch 33/100 - Train Loss: 0.049456


Epoch 34/100 - Train Loss: 0.049283


Epoch 35/100 - Train Loss: 0.049072


Epoch 36/100 - Train Loss: 0.048867


Training:  90%|███████████████████████████████████████████████████████▉      | 1975/2187 [00:43<00:04, 45.52it/s]

In [4]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
import rasterio
from rasterio.transform import Affine
from rasterio.crs import CRS

device = torch.device( 'cuda' if torch.cuda.is_available() else 'cpu' )
epochs = 100

output_base_dir = "../../data/high_resolution_data" #결과 저장할 디렉토리
# output_base_dir = "./" #결과 저장할 디렉토리

os.makedirs(output_base_dir, exist_ok=True)

meta_data_dict = {
    'jiri_1' : {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 371940.0,
       0.0, -2.5, 3915480.0)},

    'jiri_2': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 380940.0,
       0.0, -2.5, 3915480.0)},

    'sobaek': {'driver': 'GTiff', 'dtype': 'float32', 'nodata': 0.0, 'width': 3000, 'height': 3000, 'count': 6, 'crs': CRS.from_epsg(32652), 'transform': Affine(2.5, 0.0, 449760.0,
       0.0, -2.5, 4096740.0)}}

org_patch_size = 32
org_stride = 8 # (456(원래 데이터 크기) - 32) % stride == 0 이어야함

df = pd.read_csv("../../data/super_resolution_data_scaled/percentile_values.csv")

# 전체 해상도 설정
full_size = (6, 3648, 3648) # (채널, 높이, 너비)
patch_size = 8*org_patch_size # 패딩 제거 후 남는 유효 영역
stride = 8*org_stride # 데이터셋의 stride 값


for period in range(12,13,1):
    print(f"{period} 시작")
    
    #데이터 셋 만들기
    train_dataset = SuperResolutionDataset(f"../../data/super_resolution_data_scaled/{period}", stride=2)
    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    
    #모델 생성
    model = SuperResolutionResNet(num_blocks=25)
    total_parameters = sum(p.numel() for p in model.parameters())
    print(f"전체 파라미터 수: {total_parameters}")
    
    #학습
    train(model, train_loader, None, epochs=epochs, lr=1e-4, device=device)
    
    #메모리 확보
    del train_loader, train_dataset
    torch.cuda.empty_cache()
    
    for region in ['jiri_1', 'jiri_2', 'sobaek']:
        # CPU에서 처리할 NumPy 배열로 변경
        total_output = np.zeros(full_size, dtype=np.float32)
        total_count = np.zeros((1, full_size[1], full_size[2]), dtype=np.float32)
        high_resolution_data = np.zeros(full_size, dtype=np.float32) #최종 결과

        # 추론용 데이터 로더 준비
        inference_dataset = SuperResolutionDataset(
            f"../../data/super_resolution_data_scaled/{period}",
            patch_size=org_patch_size,
            s2_160m_name='s2_20.tif', s2_80m_name='s2_10.tif',
            planet_20m_name='planet.tif', label_name='planet.tif',
            s2_160m_mask_name='mask_20.tif', s2_80m_mask_name='mask_10.tif',
            stride=org_stride, regions=[region]
        )

        inference_loader = DataLoader(inference_dataset, batch_size=1, shuffle=False)

        # 시작 좌표 설정
        patch_idx = 0

        for data in tqdm(inference_loader, desc='inferencing', leave=False):
            x, _ = data
            s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = x
            s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m = (
                s2_160m.to(device), s2_160m_mask.to(device), s2_80m.to(device),
                s2_80m_mask.to(device), planet_20m.to(device)
            )

            # 모델 추론
            output = model(s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m)
            output = output[0, :, 64:320, 64:320].detach().cpu().numpy()

            y = (patch_idx // ((full_size[1] - patch_size) // stride + 1)) * stride
            x = (patch_idx % ((full_size[2] - patch_size) // stride + 1)) * stride

            total_output[:, y:y+patch_size, x:x+patch_size] += output
            total_count[:, y:y+patch_size, x:x+patch_size] += 1
            
            del s2_160m, s2_160m_mask, s2_80m, s2_80m_mask, planet_20m, output
            torch.cuda.empty_cache()
            
            patch_idx+=1

        total_output = total_output / total_count #겹쳐진 수 대로 나눔

        high_resolution_data = np.zeros(full_size, dtype=np.float32)
        for band in range(total_output.shape[0]):
            p_1st = df[(df['period'] == period) & (df['image_type'] == 's2_20') & (df['band'] == band)]['1st_percentile'].values[0].astype('float32')
            p_99th = df[(df['period'] == period) & (df['image_type'] == 's2_20') & (df['band'] == band)]['99th_percentile'].values[0].astype('float32')
            r = p_99th - p_1st
            high_resolution_data[band, :, :] = (total_output[band, :, :]* r) + p_1st

        high_resolution_data = high_resolution_data[:,24:3624, 24:3624] #패딩 걷어 내기

        # 해당 지역의 메타데이터 불러오기
        meta = meta_data_dict[region]

        # 저장을 위한 메타데이터 업데이트
        meta.update({
            'height': high_resolution_data.shape[1],
            'width': high_resolution_data.shape[2],
            'count': high_resolution_data.shape[0],
            'dtype': 'float32'
        })

        # 저장할 파일 경로 설정
        region_dir = os.path.join(output_base_dir, region)
        os.makedirs(region_dir, exist_ok=True)
        
        period_dir = os.path.join(region_dir, f"{period}")
        os.makedirs(period_dir, exist_ok=True)
        
        output_tiff_path = os.path.join(period_dir, 's2.tif')

        # TIFF 파일로 저장
        with rasterio.open(output_tiff_path, 'w', **meta) as dst:
            dst.write(high_resolution_data)
            
        print(f"{output_tiff_path}저장 완료")
        
        del total_output, total_count, high_resolution_data
        torch.cuda.empty_cache()
        
    del inference_loader, inference_dataset, model
    torch.cuda.empty_cache()


12 시작
전체 파라미터 수: 154021


Epoch 1/100 - Train Loss: 0.173151


Epoch 2/100 - Train Loss: 0.096013


Epoch 3/100 - Train Loss: 0.082908


Epoch 4/100 - Train Loss: 0.077360


...

Epoch 99/100 - Train Loss: 0.043320


Epoch 100/100 - Train Loss: 0.043306


../../data/high_resolution_data/jiri_1/12/s2.tif저장 완료


../../data/high_resolution_data/jiri_2/12/s2.tif저장 완료


../../data/high_resolution_data/sobaek/12/s2.tif저장 완료
